In [ ]:
#3rd party libraries
#!pip install pandas_bokeh
#!pip install bayesian-optimization
#!pip install six
#!pip install eli5
#!pip install yellowbrick
#!pip install scikit-optimize

In [ ]:
# Libraries
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, RandomizedSearchCV,ParameterGrid
from sklearn.linear_model import Perceptron, LogisticRegressionCV, RidgeClassifierCV, SGDClassifier, PassiveAggressiveClassifier, RidgeClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score,mean_absolute_error, confusion_matrix
from sklearn.metrics import roc_auc_score,roc_curve, auc, classification_report,precision_score,recall_score,log_loss,f1_score
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from scipy.stats import uniform, randint
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, BaggingClassifier, VotingClassifier, AdaBoostClassifier
from bayes_opt import BayesianOptimization
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from sklearn import tree
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, LabelEncoder, OneHotEncoder, MaxAbsScaler, RobustScaler, QuantileTransformer
from sklearn.svm import SVC
from sklearn import tree
import pandas_bokeh
from sklearn.decomposition import PCA
from sklearn.calibration import CalibratedClassifierCV
from numpy import mean, std
import pandas.testing as tm
from scipy import stats
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.inspection import permutation_importance
from skopt import BayesSearchCV

# Pipelines
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer

#other
from math import sqrt
import inspect
from matplotlib.font_manager import FontProperties
from scipy.stats import loguniform, uniform

import eli5

from yellowbrick.features import Rank2D
from yellowbrick.features import PCA as PCA_YB
from yellowbrick.features.radviz import RadViz
from yellowbrick.features import pca_decomposition
from yellowbrick.features import Manifold
from yellowbrick.features import JointPlotVisualizer
from yellowbrick.classifier import ClassificationReport
from yellowbrick.classifier import PrecisionRecallCurve
from yellowbrick.classifier import ClassPredictionError
from yellowbrick.model_selection import LearningCurve
from yellowbrick.model_selection import CVScores
from yellowbrick.model_selection import FeatureImportances
from yellowbrick.features import ParallelCoordinates
from yellowbrick.model_selection import RFECV
from yellowbrick.classifier import ROCAUC

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be import

In [ ]:
DATASET_URL = "https://gist.githubusercontent.com/YHYeoh/ad1a7f7170c72d621d05a70637540152/raw/5a6059c199e2c46d2f3d258f03d93cfea98e2749/marketing_campaign.csv"
data = pd.read_csv(DATASET_URL, sep = ';')

pd.set_option('plotting.backend','pandas_bokeh')

data.fillna(method = "ffill", inplace = True)
data.isnull().values.any()

label_encoder = LabelEncoder()
enc = OneHotEncoder()
data["Education"] = label_encoder.fit_transform(data["Education"])
print(label_encoder.classes_)
# enc_df = pd.DataFrame(enc.fit_transform(data[["Marital_Status"]]).toarray())
# print(enc.get_feature_names())
# data = data.join(enc_df)

marital_status_ohe = pd.get_dummies(data["Marital_Status"],prefix="Marital")
ohe_cols = marital_status_ohe.columns
data = pd.concat([data, marital_status_ohe], axis=1)


data['enroll_year'] = pd.DatetimeIndex(data.Dt_Customer).year
data['enroll_month'] = pd.DatetimeIndex(data.Dt_Customer).month
data['enroll_day'] = pd.DatetimeIndex(data.Dt_Customer).day

data.drop(["ID", 'Dt_Customer',"Z_CostContact","Z_Revenue","Marital_Status"], axis=1, inplace=True)

categorical = ['Marital_Status']
numerical = ['Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts',
       'MntFishProducts', 'MntSweetProducts', 'MntGoldProds',
       'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases',
       'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3',
       'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2',
       'Complain', 'enroll_year', 'enroll_month', 'enroll_day']
numerical_no_bool = ['Education','Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth','enroll_day','enroll_month','enroll_year']

y = data.Response
X = data.drop(['Response'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size = 0.3)

['2n Cycle' 'Basic' 'Graduation' 'Master' 'PhD']
Index(['Year_Birth', 'Education', 'Income', 'Kidhome', 'Teenhome', 'Recency',
       'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
       'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases',
       'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
       'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5',
       'AcceptedCmp1', 'AcceptedCmp2', 'Complain', 'Marital_Absurd',
       'Marital_Alone', 'Marital_Divorced', 'Marital_Married',
       'Marital_Single', 'Marital_Together', 'Marital_Widow', 'Marital_YOLO',
       'enroll_year', 'enroll_month', 'enroll_day'],
      dtype='object')


In [ ]:
def hasmethod(obj, name):
    return inspect.ismethod(getattr(obj, name, None))

def ROC_Curve_Plot(model,X_test,y_test):
    predProb = model.predict_proba(X_test)
    preds = predProb[:,1]
    fpr, tpr, threshold = roc_curve(y_test, preds,pos_label=1)
    roc_auc = auc(fpr, tpr)
    plt.close()
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    #plt.xlim([0, 1])
    #plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    return fpr,tpr

def setupPreprocessPipeline(scaler):
	ss = Pipeline(steps=[('scaler',scaler)])
	#ohe = Pipeline(steps=[('ohe', OneHotEncoder(handle_unknown = 'ignore'))])
	preprocess = ColumnTransformer(
                    transformers=[
                        ('cont', ss, numerical_no_bool)
                        #('cat', ohe, categorical),
                        #('le', le, ordinal),
                        ],remainder='passthrough')
	return preprocess

def feature_importance(classifier, feature_names, scaler_name):
	if (hasattr(classifier,'coef_')):
		importance = classifier.coef_[0]
	elif (hasattr(classifier,'coefs_')):
		importance = classifier.coefs_
	elif (hasattr(classifier,'feature_importances_')):
		importance = classifier.feature_importances_
	else:
		print("Cannot extract feature importance, skipping")
		return

	for i,v in enumerate(importance):
		print('Feature: %d, Score: %.5f' % (i,v))
	zipped = zip(feature_names, importance)
	df = pd.DataFrame(zipped, columns=["feature", "value"])
	# Sort the features by the absolute value of their coefficient
	df["abs_value"] = df["value"].apply(lambda x: abs(x))
	df["colors"] = df["value"].apply(lambda x: "green" if x > 0 else "red")
	df = df.sort_values("abs_value", ascending=False)
	# plot feature importance
	fig, ax = plt.subplots(1, 1, figsize=(16, 9))
	sns.barplot(x="feature",
	            y="value",
	            data=df.head(20),
	           palette=df.head(20)["colors"])
	plt.gcf().subplots_adjust(bottom=0.30)
	ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=14)
	ax.set_title("Top 20 Features for {} w/ {}".format(classifier.__class__.__name__, scaler_name), fontsize=25)
	ax.set_ylabel("Coef", fontsize=22)
	ax.set_xlabel("Feature Name", fontsize=22)
	plt.show()

def evaluation(y, y_hat, title):
    cm = confusion_matrix(y, y_hat)
    precision = precision_score(y, y_hat)
    recall = recall_score(y, y_hat)
    accuracy = accuracy_score(y,y_hat)
    f1 = f1_score(y,y_hat)
    print('Recall: ', recall)
    print('Accuracy: ', accuracy)
    print('Precision: ', precision)
    print('F1: ', f1)
    sns.heatmap(cm,  cmap= 'PuBu', annot=True, fmt='g', annot_kws=    {'size':20})
    plt.xlabel('predicted', fontsize=18)
    plt.ylabel('actual', fontsize=18)
    plt.title(title, fontsize=18)
    plt.show()
    
def metrics_summary(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    accuracy=accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall =  recall_score(y_test, y_pred) #sensitivity
    specificity = tn / (tn+fp)
    g_mean= sqrt(recall * specificity)
    mse =mean_squared_error(y_test, y_pred, squared=False)
    r2=r2_score(y_test, y_pred)
    ros = roc_auc_score(y_test, y_pred)
    ll = log_loss(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    metrics_collection_dict ={
        'accuracy':['accuracy',accuracy],
        'precision':['precision',precision],
        'recall':['recall',recall],
        'specificity':['specificity',specificity],
        'g_mean':['g_mean',g_mean],
        'mean_square_error':['mean_square_error',mse],
        'r2':['r2',r2],
        'roc_auc_score':['roc_auc_score',ros],
        'log_loss':['log_loss',ll],
        'f1_score':['f1_score',f1]
    } 
    return metrics_collection_dict

In [ ]:
def cross_validate(classifier, cv):
	scalers = [StandardScaler(),MinMaxScaler(),MaxAbsScaler(), RobustScaler()]
	train_acc = []
	test_acc = []
	mean = []
	result = []
	for scaler in scalers:
		fpr = None
		tpr = None
		preprocess = setupPreprocessPipeline(scaler)
		pipeline = Pipeline(steps=[
	        ('preprocess', preprocess),
	        ('classifier', classifier)
    	])

		train_acc = []
		test_acc = []
		mean = []

		for train_ind, val_ind in cv.split(X_train, y_train):
			X_t, y_t = X_train.iloc[train_ind], y_train.iloc[train_ind]
			pipeline.fit(X_t, y_t)
			y_hat_t = pipeline.predict(X_t)
			train_acc.append(accuracy_score(y_t, y_hat_t))
			X_val, y_val = X_train.iloc[val_ind], y_train.iloc[val_ind] 
			y_hat_val = pipeline.predict(X_val)
			test_acc.append(accuracy_score(y_val, y_hat_val))

		#ohe_cols = list(pipeline.named_steps['preprocess'].named_transformers_['cat'].named_steps['ohe'].get_feature_names(input_features=categorical))
		numeric_feature_list = list(numerical)
		for i in ohe_cols:
			numeric_feature_list.append(i)
		print(len(numeric_feature_list))
		evaluation(y_val, y_hat_val, 'Confusion Matrix {} + {}'.format(classifier.__class__.__name__, scaler.__class__.__name__).strip())
		print('Mean Training Accuracy: {} | Standard Deviation: {}'.format(np.mean(train_acc),np.std(test_acc)))
		print('Mean Validation Accuracy: {} | Standard Deviation: {}'.format(np.mean(test_acc),np.std(test_acc)))
		print('\n')
        
		feature_importance(classifier, numeric_feature_list, scaler.__class__.__name__ )
		metrics_summ = metrics_summary(y_val,y_hat_val)
		if hasmethod(pipeline['classifier'], 'predict_proba'):
			fpr,tpr = ROC_Curve_Plot(pipeline,X_val,y_val)
		result.append({
            'classifier':classifier.__class__.__name__,
            'scalerName':scaler.__class__.__name__,
            'metrics_summ':metrics_summ,
            'fpr':fpr,
            'tpr':tpr
        })
	return result

In [ ]:
models = [ 
	LogisticRegressionCV(max_iter= 1200), 
	RidgeClassifierCV(),
	SVC(kernel = 'linear',max_iter= -1), 
	Perceptron(),
	PassiveAggressiveClassifier(), 
	DecisionTreeClassifier(), #no coef 
	KNeighborsClassifier(),#no feat_import, use permutation_importance 
	GaussianNB(), #no feat_import, use permutation_importance 
	LGBMClassifier(),#no coef 
	RandomForestClassifier(), #no coef 
	GradientBoostingClassifier(),#no coef 
	PassiveAggressiveClassifier(), 
	ExtraTreesClassifier(), #no coef 
	XGBClassifier(),
	AdaBoostClassifier(), #no coef 
	]

model_result = []

for model in models:
	print(model.__class__.__name__)
	model_result.append(cross_validate(model,KFold()))

In [ ]:
standardScalerList = []
minMaxScalerList = []
maxAbsScalerList = []
robustScalerList = []
for collection in model_result:
    standard = collection[0]
    standardScalerList.append({'classifier':standard['classifier'],'metrics_summ':standard['metrics_summ'],'fpr':standard['fpr'],'tpr':standard['tpr'] })
    minMax = collection[1]
    minMaxScalerList.append({'classifier':minMax['classifier'],'metrics_summ':minMax['metrics_summ'],'fpr':minMax['fpr'],'tpr':minMax['tpr'] })
    maxAbs = collection[2]
    maxAbsScalerList.append({'classifier':maxAbs['classifier'],'metrics_summ':maxAbs['metrics_summ'],'fpr':maxAbs['fpr'],'tpr':maxAbs['tpr'] })
    robust = collection[3]
    robustScalerList.append({'classifier':robust['classifier'],'metrics_summ':robust['metrics_summ'],'fpr':robust['fpr'],'tpr':robust['tpr'] })


In [ ]:
nameList = []

for object in model_result:
    nameList.append(object[0]['classifier'])
metric_list = ['accuracy','precision','recall','specificity','g_mean'
                   ,'mean_square_error','r2','roc_auc_score','log_loss','f1_score']
scaler = ['Standard Scaler','Min Max Scaler','Max Abs Scaler','robust Scaler']
for metric in metric_list:
    resultList = []
    for model in model_result:
        resultList.append(model[0]['metrics_summ'][metric][1])
        
    accDF = pd.DataFrame(list(zip(nameList,resultList)),columns=['trained_model',metric])
    plt.title("Models' "+metric+ " with Standard Scaler")
    ax = sns.barplot(data=accDF.sort_values(metric,ascending=False),orient='h',palette ="Paired" , y = 'trained_model',x=metric)
    plt.show()

In [ ]:
for metric in metric_list:
    resultList = []
    for model in model_result:
        resultList.append(model[1]['metrics_summ'][metric][1])
        
    accDF = pd.DataFrame(list(zip(nameList,resultList)),columns=['trained_model',metric])
    plt.title("Models' "+metric+ " with Min Max Scaler")
    ax = sns.barplot(data=accDF.sort_values(metric,ascending=False),orient='h',palette ="Paired" , y = 'trained_model',x=metric)
    plt.show()

In [ ]:
for metric in metric_list:
    resultList = []
    for model in model_result:
        resultList.append(model[2]['metrics_summ'][metric][1])
        
    accDF = pd.DataFrame(list(zip(nameList,resultList)),columns=['trained_model',metric])
    plt.title("Models' "+metric+ " with Max Abs Scaler")
    ax = sns.barplot(data=accDF.sort_values(metric,ascending=False),orient='h',palette ="Paired" , y = 'trained_model',x=metric)
    plt.show()

In [ ]:

for metric in metric_list:
    resultList = []
    for model in model_result:
        resultList.append(model[3]['metrics_summ'][metric][1])
        
    accDF = pd.DataFrame(list(zip(nameList,resultList)),columns=['trained_model',metric])
    plt.title("Models' "+metric+ " with Robust Scaler")
    ax = sns.barplot(data=accDF.sort_values(metric,ascending=False),orient='h',palette ="Paired" , y = 'trained_model',x=metric)
    plt.show()

In [ ]:
#Grid search params
scoring = 'f1'
fold=10
featureNumList = list(range(1,X_train.shape[1]))
modelsWithParam = [
    {
        'model':GradientBoostingClassifier(),'param': {
        'loss':['deviance', 'exponential'],
        'learning_rate': [0.1,0.01,0.005,0.001,0.0005,0.0001],
        'min_samples_split': [0.1, 0.5, 5.0,10.0,20.0,40.0,80.0],
        'min_samples_leaf': [0.1, 0.5, 5,10,20,40,80],
        'max_depth':[5,10, 20, 30, 40, 60, 100, None],
        'max_features':featureNumList,
        'subsample':['0.8','1','1.2','1.4','1.6'],
        'random_state': [42]
        }}
    ,{
        'model':RandomForestClassifier(),'param':{'bootstrap': [True, False],
        'max_depth': [5,10, 20, 30, 40, 60, 100, None],
        'max_features': featureNumList,
        'min_samples_leaf': [1, 2, 4],
        'min_samples_split': [0.1, 0.5, 5.0,10.0,20.0,40.0,80.0],
        'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
        'random_state': [42]
        }
},{
        'model':LGBMClassifier(),'param':{
        'learning_rate': [0.1,0.01,0.005,0.001,0.0005,0.0001],
        'n_estimators': [1, 2, 4, 8, 16, 32, 64, 100, 200],
        'max_depth': [5,10, 20, 30, 40, 60, 100, None],
        'min_samples_split': [0.1, 0.5, 5.0,10.0,20.0,40.0,80.0],
        'min_samples_leaf':[0.1, 0.5, 5,10,20,40,80],
        'random_state': [42]
    }},{
            
        'model':SVC(),'param':{
        'C': [0.1, 1, 10, 100, 1000],  
        'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
        'kernel': ['linear', 'rbf', 'sigmoid', 'precomputed'],
        'random_state': [42] 
    }},
    {
        'model':SVC(),'param':{
        'C': [0.1, 1, 10, 100, 1000],  
        'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
        'kernel': ['poly'],
        'degree':[3,4,5,6],
        'random_state': [42] 
    }},{
        'model':RidgeClassifier(),'param':{'alpha': [100,10,1, 0.1,0.05,0.001,0.0005],#100,10,1, 0.1, 0.01, 0.001
                                 'fit_intercept':[True,False],
                                 'normalize':[True,False],
                                 'copy_X':[True],
                                 'max_iter':[1,5,10,20,40,80,160,320,740,1480],#[50,100,500,1000,2000,4000,8000]
                                 'solver':['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
                                 'random_state':[42]
        }}]

In [ ]:
#gridsearchcv
def modelBestFit(item):
    model = item['model']
    paramGrid = item['param']
    search = GridSearchCV(model, paramGrid,n_jobs=-1,pre_dispatch='1*n_jobs'
    ,scoring = scoring,refit = True,cv=fold)
    search.fit(X_train,y_train)
    best_score =search.score(X_test,y_test)
    model_name = model.__class__.__name__
    return {'model_name':model_name,'best_score':best_score,'best_model':search.best_estimator_}

def bestModel(modelsAndParams):
    modelPerformance = pd.DataFrame()
    for item in modelsAndParams:
        result = modelBestFit(item)
        
        modelPerformance = modelPerformance.append(result,ignore_index=True)
        
    modelPerformance.sort_values(by='best_score',ascending=False,inplace=True)
    return modelPerformance
result = bestModel(modelsWithParam)

In [ ]:
result

In [ ]:
#randomised search and bayesian search params
scoring = 'f1'
fold = 10
featureNumList = list(range(1,X_train.shape[1]))
modelsWithParam = [ {
        'model':GradientBoostingClassifier(),'param': {
        'loss':['deviance', 'exponential'],
        'learning_rate': loguniform(1e-5, 100),
        'min_samples_split': loguniform(1e-5, 100),
        'min_samples_leaf': loguniform(1e-5, 100),
        'max_depth':list(range(5,100)),
        'max_features':featureNumList,
        'subsample':loguniform(0.1, 2),
        'random_state': [42]
        }}
    ,
   {
        'model':RandomForestClassifier(),'param':{'bootstrap': [True, False],
        'max_depth': list(range(5,100)),
        'max_features': featureNumList,
        'min_samples_leaf': loguniform(1e-5, 100),
        'min_samples_split': loguniform(1e-5, 100),
        'n_estimators': list(range(1,5000)),
        'random_state': [42]
        }
},{
        'model':LGBMClassifier(),'param':{
        'learning_rate': loguniform(1e-5, 100),
        'n_estimators': list(range(1,200)),
        'max_depth': list(range(5,100)),
        'min_samples_split': loguniform(1e-5, 100),
        'min_samples_leaf':loguniform(1e-5, 100),
        'random_state': [42]
    }},{
            
        'model':SVC(),'param':{
        'C': loguniform(1e-5, 10),  
        'gamma': loguniform(1e-5, 100), 
        'kernel': ['linear', 'rbf', 'sigmoid', 'precomputed'],
        'random_state': [42] 
    }},
    {
        'model':SVC(),'param':{
        'C': loguniform(1e-5, 100),  
        'gamma': loguniform(1e-5, 100), 
        'kernel': ['poly'],
        'degree':list(range(3,12)),
        'random_state': [42] 
    }},{
        'model':RidgeClassifier(),'param':{'alpha': loguniform(1e-5, 100),#100,10,1, 0.1, 0.01, 0.001
                                 'fit_intercept':[True,False],
                                 'normalize':[True,False],
                                 'copy_X':[True],
                                 'max_iter':list(range(1,5000)),#[50,100,500,1000,2000,4000,8000]
                                 'solver':['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
                                 'random_state':[42]
        }}]

In [ ]:
#randomised search cv
def modelBestFit(item):
    model = item['model']
    paramGrid = item['param']
    search = RandomizedSearchCV(model, paramGrid,n_jobs=-1,pre_dispatch='1*n_jobs',scoring = scoring,refit = True,cv=fold)
    search.fit(X_train,y_train)
    best_score =search.score(X_test,y_test)
    model_name = model.__class__.__name__
    return {'model_name':model_name,'best_score':best_score,'best_model':search.best_estimator_}

def bestModel(modelsAndParams):
    modelPerformance = pd.DataFrame()
    for item in modelsAndParams:
        result = modelBestFit(item)
        
        modelPerformance = modelPerformance.append(result,ignore_index=True)
        
    modelPerformance.sort_values(by='best_score',ascending=False,inplace=True)
    return modelPerformance
 
result = bestModel(modelsWithParam)

In [ ]:
result

In [ ]:
#bayesian search cv
def modelBestFit(item):
    model = item['model']
    paramGrid = item['param']
    search = BayesSearchCV(model, paramGrid,n_jobs=-1,pre_dispatch='1*n_jobs'
                           ,scoring = scoring,refit = True,cv = fold)
    search.fit(X_train,y_train)
    best_score =search.score(X_test,y_test)
    model_name = model.__class__.__name__
    return {'model_name':model_name,'best_score':best_score,'best_model':search.best_estimator_}

def bestModel(modelsAndParams):
    modelPerformance = pd.DataFrame()
    for item in modelsAndParams:
        result = modelBestFit(item)
        
        modelPerformance = modelPerformance.append(result,ignore_index=True)
        
    modelPerformance.sort_values(by='best_score',ascending=False,inplace=True)
    return modelPerformance
 
result = bestModel(modelsWithParam)

In [ ]:
result